In [1]:
import pandas as pd

In [2]:
# Funzione che si occupa di sovrascrivere un dataframe per selezione e rinomina colonne

def select_and_rename_columns(df, cols, names):
    res = df[cols].copy()
    res = res.rename(columns=names).copy()
    return res

In [3]:
# Funzione che cambia l'item type

def replace_item_type(df, col, values):
    df[col] = df[col].replace(values)
    return df

In [4]:
# Funzione che elimina tutti i valori NaN

def fill_na(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].fillna('')
    return df

In [5]:
# Funzione che aggiunge una nuova colonna per ogni IC ed EC e imposta si o no come valore se l'inclusion/exclusion criteria è presente nelle note

def add_and_fill_columns(df, nincl, nexcl, suffix, colnotes):
    criteria = []
    for i in range(1,nincl+1):
        col = 'IC'+str(i)+' '+suffix
        criteria.append(col)
        df[col] = pd.NA
    
    for e in range(1,nexcl+1):
        col = 'EC'+str(e)+' '+suffix
        criteria.append(col)
        df[col] = pd.NA

    for index, row in df.iterrows():
        notes = row[colnotes]
        for c in criteria:
            crit = c.split(' ')[0]
            if crit in notes:
                df.at[index, c] = 'yes'
            else:
                df.at[index, c] = 'no'
    return df

In [6]:
def convert_paper_inclusion(df, column_name, new_column_name):
    df[new_column_name] = df[column_name].apply(lambda x: 'Include' if x == 1.0 else 'Exclude' if x == 0.0 else 'Rejected by ASReview')
    return df

In [7]:
def create_clean_df(df, cols, names, itype_colname, colvalues, nincl, nexcl, suffix, colnotes, inclcol, newcolname):
    df = select_and_rename_columns(df, cols, names)
    df = replace_item_type(df, itype_colname, colvalues)
    df = fill_na(df)
    df = add_and_fill_columns(df, nincl, nexcl, suffix, colnotes)
    df = convert_paper_inclusion(df, inclcol, newcolname)
    return df

In [8]:
def join_two_datasets(df1, df2, join_column):
    # Merge the datasets on the predefined column
    merged_df = pd.merge(df1, df2, on=join_column, suffixes=('_df1', '_df2'))

    # Eliminate repeated columns
    for column in merged_df.columns:
        if column.endswith('_df1') or column.endswith('_df2'):
            base_column = column.split('_')[0]
            if base_column in merged_df.columns:
                merged_df.drop(column, axis=1, inplace=True)
            else:
                merged_df.rename(columns={column: base_column}, inplace=True)

    return merged_df

In [9]:
# Common parameters
colname = 'Item Type'
colvalues = {'CONF': 'conferencePaper', 'JOUR': 'journalArticle'}
nincl = 6 # Number of inclusion criteria
nexcl = 5 # Number of exclusion criteria

In [10]:
# Dataframe Federico

dataF = pd.read_csv(r'C:\Users\ROBECAPU\Desktop\SLR Federico\Federico.csv')
colsfl = ['record_id', 'type_of_reference', 'title', 'authors', 'abstract', 'doi', 'exported_notes_1', 'included', 'asreview_ranking'] # Columns to keep
namesfl = {'type_of_reference':'Item Type', 'title':'Title', 'authors':'Authors', 'doi':'DOI', 'abstract':'Abstract', 'included':'Paper Inclusion Federico', 'exported_notes_1':'Notes Federico', 'asreview_ranking':'Ranking Federico'} # New names for the columns
suffix = 'Federico' # Suffix for the inclusion/exclusion criteria columns
colnotes = 'Notes Federico' # Column containing the notes
inclcol = 'Paper Inclusion Federico' # Column containing the inclusion/exclusion decision
newcolname = 'Decision Federico' # New column name for the inclusion/exclusion decision

dff = create_clean_df(dataF, colsfl, namesfl, colname, colvalues, nincl, nexcl, suffix, colnotes, inclcol, newcolname) # Cleaned dataframe
dff.to_csv(r'C:\Users\ROBECAPU\Desktop\SLR Federico\Federico_clean.csv', index=False)

In [11]:
# Dataframe Lorenzo

dataL = pd.read_csv(r'C:\Users\ROBECAPU\Desktop\SLR Federico\Lorenzo.csv')
colsfl = ['record_id', 'type_of_reference', 'title', 'authors', 'abstract', 'doi', 'exported_notes_1', 'included', 'asreview_ranking']
namesfl = {'type_of_reference':'Item Type', 'title':'Title', 'authors':'Authors', 'doi':'DOI', 'abstract':'Abstract', 'included':'Paper Inclusion Lorenzo', 'exported_notes_1':'Notes Lorenzo', 'asreview_ranking':'Ranking Lorenzo'}
suffix = 'Lorenzo'
colnotes = 'Notes Lorenzo'
inclcol = 'Paper Inclusion Lorenzo' # Column containing the inclusion/exclusion decision
newcolname = 'Decision Lorenzo' # New column name for the inclusion/exclusion decision

dfl = create_clean_df(dataL, colsfl, namesfl, colname, colvalues, nincl, nexcl, suffix, colnotes, inclcol, newcolname) # Cleaned dataframe
dfl.to_csv(r'C:\Users\ROBECAPU\Desktop\SLR Federico\Lorenzo_clean.csv', index=False)

In [12]:
# Dataframe Beatrice

dataB = pd.read_csv(r'C:\Users\ROBECAPU\Desktop\SLR Federico\Beatrice.csv')
colsb = ['record_id', 'Item Type', 'Title', 'Author', 'Abstract Note', 'DOI', 'exported_notes_1', 'included', 'asreview_ranking']
namesb = {'Author':'Authors', 'included':'Paper Inclusion Beatrice', 'exported_notes_1':'Notes Beatrice', 'Abstract Note':'Abstract', 'asreview_ranking':'Ranking Beatrice'}
suffix = 'Beatrice'
colnotes = 'Notes Beatrice'
inclcol = 'Paper Inclusion Beatrice' # Column containing the inclusion/exclusion decision
newcolname = 'Decision Beatrice' # New column name for the inclusion/exclusion decision

dfb = create_clean_df(dataB, colsb, namesb, colname, colvalues, nincl, nexcl, suffix, colnotes, inclcol, newcolname) # Cleaned dataframe
dfb.to_csv(r'C:\Users\ROBECAPU\Desktop\SLR Federico\Beatrice_clean.csv', index=False)

In [13]:
index = 'record_id'

final = join_two_datasets(dff, dfl, index)

In [14]:
final = join_two_datasets(final, dfb, index)

In [15]:
# Move the specified columns to the end of the dataframe
cols_to_move = ['Decision Federico', 'Decision Lorenzo', 'Decision Beatrice']
final = final[[col for col in final if col not in cols_to_move] + cols_to_move]

In [16]:
def eliminate_columns_not_in_list(df, columns_to_keep):
    return df[columns_to_keep]

final = eliminate_columns_not_in_list(final, ['record_id', 'Item Type', 'Title', 'Authors', 'DOI', 'Notes Federico', 'Notes Lorenzo', 'Notes Beatrice', 'Decision Federico', 'Decision Lorenzo', 'Decision Beatrice'])

In [17]:
def calculate_final_decision(row):
    decisions = [row['Decision Federico'], row['Decision Lorenzo'], row['Decision Beatrice']]
    
    if all(decision == 'Include' for decision in decisions):  # If all decisions are 'Include'
        return 'Include'
    elif all(decision in ['Exclude', 'Rejected by ASReview'] for decision in decisions):  # If all decisions are 'Exclude' or 'Rejected by ASReview'
        return 'Exclude'
    elif row['Decision Federico'] == 'Include' and all(decision in ['Exclude', 'Rejected by ASReview'] for decision in decisions[1:]):  # If Federico says 'Include' and the others say 'Exclude' or 'Rejected by ASReview'
        return 'Maybe'
    elif row['Decision Federico'] in ['Exclude', 'Rejected by ASReview'] and 'Include' in decisions[1:]:  # If Federico says 'Exclude' or 'Rejected by ASReview' and at least one of the others says 'Include'
        return 'Maybe'
    elif row['Decision Federico'] == 'Include' and 'Include' in decisions[1:]:  # If Federico says 'Include' and at least one of the others says 'Include'
        return 'Include'
    else:
        return 'Maybe'

In [18]:
def determine_final_decision(row):
    if row['Intermediate Decision'] == 'Include':
        return 'Accept'
    elif row['Intermediate Decision'] == 'Exclude':
        return 'Reject'
    else:
        return ''

In [19]:
final = eliminate_columns_not_in_list(final, ['record_id', 'Item Type', 'Title', 'Authors', 'DOI', 'Notes Federico', 'Notes Lorenzo', 'Notes Beatrice', 'Decision Federico', 'Decision Lorenzo', 'Decision Beatrice'])
final['Intermediate Decision'] = final.apply(calculate_final_decision, axis=1)
final['Final Decision'] = final.apply(determine_final_decision, axis=1)
final.to_csv(r'C:\Users\ROBECAPU\Desktop\SLR Federico\Final.csv', index=False)